### This notebook is for beginners who want to know about hyperparameter optimization

### What is hyperparameter tuning
Hyperparameter tuning or optimization is the method to find optimal parameters to your machine learning model

We are going to use [lightGBM](https://lightgbm.readthedocs.io/en/latest/) as a machine learning model. And we are going to use [Optuna](https://optuna.readthedocs.io/en/stable/) which is auto hyperparameter tuner to tune the parameters of LGBM.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/jane-street-market-prediction/train.csv',nrows=30000)


In [ ]:
features = [col for col in list(train.columns) if 'feature' in col]
train = train[train['weight'] != 0]
train['action'] = (train['resp'].values > 0).astype(int)
f_mean = train.mean()
train = train.fillna(f_mean)
X = train.loc[:, features]
y = train.loc[:, 'action']
del train
X = np.array(X)
y = np.array(y)

In [ ]:
import lightgbm as lgb
import optuna
from sklearn.model_selection import train_test_split
import sklearn

In [ ]:
import optuna
def objective(trial):
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25)
    dtrain = lgb.Dataset(X_train, label=y_train)

    param = {
        "objective": "binary",
        "metric": "binary_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }

    gbm = lgb.train(param, dtrain)
    preds = gbm.predict(X_test)
    pred_labels = np.rint(preds)
    accuracy = sklearn.metrics.accuracy_score(y_test, pred_labels)
    return accuracy

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

In [ ]:
print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

### So these are the best parameters for your models.